In [95]:
import pandas as pd

In [96]:
position_option = pd.read_csv('./data/position_option.csv').drop_duplicates()
position_option = position_option[["time", "symbol", "contractRight", "lastTradeDateOrContractMonth", "strike", "delta", "position"]]

In [97]:
# get the latest record with unique symbol secType contractRight lastTradeDateOrContractMonth strike

comparator = position_option.groupby(["symbol", "contractRight", "lastTradeDateOrContractMonth", "strike"]).agg({'time': 'max'}).reset_index()

# get the delta of the latest record with unique symbol secType contractRight lastTradeDateOrContractMonth strike

order_calculator = position_option.merge(comparator, on=['symbol', "contractRight",'time'], how='inner')

# sum the delta of same symbol

order_calculator.groupby(['symbol']).agg({'delta': 'sum'}).reset_index().to_dict('records')





[{'symbol': 'AAPL', 'delta': 0.6598343237975796},
 {'symbol': 'NFLX', 'delta': 0.002096144017257884},
 {'symbol': 'TSLA', 'delta': 0.583328661752576}]

In [98]:
order_calculator

,time,symbol,contractRight,lastTradeDateOrContractMonth_x,strike_x,delta,position,lastTradeDateOrContractMonth_y,strike_y
0,2023-10-04 12:20:36,AAPL,C,20231020,170.0,0.659834,-1.0,20231020,170.0
1,2023-10-04 12:20:38,NFLX,C,20231020,382.5,0.502171,1.0,20231020,382.5
2,2023-10-04 12:20:38,TSLA,C,20231020,252.5,0.583329,0.0,20231020,252.5
3,2023-10-04 12:20:39,NFLX,P,20231020,382.5,-0.500074,-2.0,20231020,382.5


In [99]:
comparator

,symbol,contractRight,lastTradeDateOrContractMonth,strike,time
0,AAPL,C,20231020,170.0,2023-10-04 12:20:36
1,NFLX,C,20231020,382.5,2023-10-04 12:20:38
2,NFLX,P,20231020,382.5,2023-10-04 12:20:39
3,TSLA,C,20231020,252.5,2023-10-04 12:20:38


In [100]:
position_option

,time,symbol,contractRight,lastTradeDateOrContractMonth,strike,delta,position
0,2023-10-04 12:20:36,AAPL,C,20231020,170.0,0.659834,-1.0
1,2023-10-04 12:20:38,NFLX,C,20231020,382.5,0.502171,1.0
2,2023-10-04 12:20:38,TSLA,C,20231020,252.5,0.583329,0.0
3,2023-10-04 12:20:39,NFLX,P,20231020,382.5,-0.500074,-2.0


In [101]:
import json
with open('input.json', 'r') as f:
    config = json.load(f)

In [102]:
position_option["targe_delta"] = position_option.apply(lambda x: config['trading'][x['symbol']]['target_delta'] if (x['symbol'] in config['trading']) else 0, axis=1)

In [103]:
# position_option = position_option.groupby(['symbol']).agg({'delta': 'sum', 'position':'sum'}).reset_index()
position_option["targe_delta"] = position_option.apply(lambda x: config['trading'][x['symbol']]['target_delta'] if (x['symbol'] in config['trading']) else 0, axis=1)
position_option["idealPosition"] = round((position_option["targe_delta"] - position_option["delta"]) * position_option["position"] * 100)

In [104]:
position_option

,time,symbol,contractRight,lastTradeDateOrContractMonth,strike,delta,position,targe_delta,idealPosition
0,2023-10-04 12:20:36,AAPL,C,20231020,170.0,0.659834,-1.0,0,66.0
1,2023-10-04 12:20:38,NFLX,C,20231020,382.5,0.502171,1.0,0,-50.0
2,2023-10-04 12:20:38,TSLA,C,20231020,252.5,0.583329,0.0,0,-0.0
3,2023-10-04 12:20:39,NFLX,P,20231020,382.5,-0.500074,-2.0,0,-100.0


: 